<h1>  Build agentic AI application using Langgraph </h1>
<p> Built by Jakob Lindstrøm, aka DataJakob@Github,  for Lumos  SDC. </p>
<p> In this notebook  we will create a agentic chatbot that is especially good at Q&A for finance and portfolio generation </p> 

<h3> 1 Import libraries </h3>

In [37]:
# Enviroment 
import os

# "Lang" packages
import langchain
import langgraph
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
# Other

# .env variables
# my_api_key = os.getenv("OPENAI_API_KEY")

<h2> 2 State and classes </h2>

In [38]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    mission: str
    company_info: str
    stocks: Annotated[list, add_messages]
    optimal_portfolio: Annotated


class Mission(BaseModel):
    mission: str = Field(description="""
                         Answer with one word based on the user query:
                         - "append", if the user wants to append  the stock into the portfolio.
                         - "more_info"  if the user wants more info about a stock or something else.
                         - "analyze"  if  the user wants to analyze the current portfolio,
                         - "other", if nothing of the above is specified.
                        """)
    
class Filter(BaseModel):
    ticker: str = Field(description="""
                        Based on the  previous message, I want you to filter down the message to a stock ticker from Oslo Børs.
                        - 1. First you need to identify the company inn question
                        - 2. Reduce it into its ticker name
                        - 3. Then add .OL to it, such that it can be detected by Yahoo Finance

                        For example if the company is Equinor I want you to return only: "EQNR.OL"
                        """)

<h2> 3 Model and Nodes </h2>

In [39]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    api_key=my_api_key
)

In [40]:
a =  llm.with_structured_output(Mission)
response = a.invoke("hva gjør  equinor").mission
print(response)

more_info


In [53]:
def navigator(state: State):
    nav_model =  llm.with_structured_output(Mission)
    response = nav_model.invoke(state["messages"]).mission
    return {"mission": response}



def info(state: State):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a world class technical documentation writer."),
        ("user", "{input}")
    ])
    info_model = prompt | llm 
    response =  info_model.invoke({"input": state.messages})
    state["company_info"] = response.content


    
def stock_appender(state: State):
    nav_model =  llm.with_structured_output(Filter)
    response = nav_model.invoke(state["messages"]).ticker
    state["mission"] = response



def portfolio_analyzer(state: State):
    # Call the analyzer function: analyze(state["stocks"])
    state["messages"] = 0# what-i-return



def final_response(state: State):
    if state["mission"] == "other":
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are chatbot specializing in financial advisory. 
             The user has given you a unrelated query, make the user give queries about appending stocks to a porfolio,
             get information about a company or analyze a portfolio """),
            ("user", "{input}")
        ])
        final_model = prompt | llm 
        response =  final_model.invoke({"input": state["messages"]})
        return {"messages": response.content}

    elif state["mission"] == "analyze":
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are chatbot specializing in financial advisory. 
             Use the latest query with stocks and its position to inform the user about the optimal portfolio according to the Sharpe ratio."""),
            ("user", "{input}")
        ])
        final_model = prompt | llm 
        response =  final_model.invoke({"input": state["messages"]})
        return {"messages": response.content}
    
    elif state["mission"] == "append":
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are chatbot specializing in financial advisory. 
             You have just added a stock to a portfolio, ask the user if a new stock should be added.
             Or if the user wants to get info about a company or  analyze the portfolio"""),
            ("user", "{input}")
        ])
        final_model = prompt | llm 
        response =  final_model.invoke({"input": state["messages"]})
        return {"messages":  response.content}
        
    else:
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are chatbot specializing in financial advisory. 
             Give key pieces of information about a company that the user have requested."""),
            ("user", "{input}")
        ])
        final_model = prompt | llm 
        response =  final_model.invoke({"input": state["messages"]})
        return {"messages": response.content}

<h2> 4 Edges</h2>

In [54]:
def decision(state: State):

    if state["mission"] == "append":
        return "add_stock"
    elif state["mission"] == "analyze":
        return  "analyze_portfolio"
    elif state["mission"] == "info":
        return "more_info"
    else:
        return "other"

<h2> 5 Workflow Compilation </h2>

In [55]:
workflow = StateGraph(State)
# Nodes for something.
workflow.add_node("nav", navigator)
workflow.add_node("info", info)
workflow.add_node("append", stock_appender)
workflow.add_node("analyze", portfolio_analyzer)
workflow.add_node("response", final_response)

workflow.add_edge(START, "nav")     # Non-optional move
workflow.add_conditional_edges("nav",   # Starting node
                               decision,    # Decider for which node to go to
                               # Options
                               {"more_info":"info",
                                "add_stock":"append",
                                "analyze_portfolio":"analyze",
                                "other":"response"}

)
# Non-optional moves
workflow.add_edge("info", "response")
workflow.add_edge("append", "response")
workflow.add_edge("analyze", "response")
workflow.add_edge("response", END)

mygraph = workflow.compile()

<h2> 6 Start chatting :) </h2>

In [56]:
def stream_graph_updates(user_input: str):
    for event in mygraph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"    
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: {'mission': 'other'}
Assistant: {'messages': 'Hello! If you have any questions about investing, such as how to append stocks to your portfolio, get information about a specific company, or analyze your current portfolio, feel free to ask!'}
Assistant: {'mission': 'more_info'}
Assistant: {'messages': 'Equinor ASA is a Norwegian multinational energy company primarily focused on oil and gas exploration and production, as well as renewable energy. Here are some key pieces of information about Equinor:\n\n1. **Foundation and Headquarters**: Equinor was founded in 1972 and is headquartered in Stavanger, Norway.\n\n2. **Formerly Known As**: The company was previously known as Statoil until it rebranded to Equinor in 2018 to reflect its commitment to renewable energy and sustainability.\n\n3. **Operations**: Equinor operates in more than 30 countries and is involved in various segments of the energy sector, including:\n   - Oil and gas exploration and production\n   - Renewable ener